log_utils.py — Reusable Logging Module

In [1]:
import os
import json
import time
import platform
import psutil
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def log_model_performance(
    model_name,
    target_variable,
    X_train,
    X_test,
    y_test,
    y_pred,
    model_object,
    training_time_sec,
    dataset_name="Final_Model_Ready_Clean_Reshaped",
    tuning_type="None",
    gpu_used="NVIDIA GeForce RTX 2050",
    extra_notes=""
):
    # Evaluation
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Inference time
    start_infer = time.time()
    _ = model_object.predict(X_test[:1])
    end_infer = time.time()
    inference_time = round((end_infer - start_infer), 6)

    # Model size
    joblib.dump(model_object, "temp_model.pkl")
    model_size_mb = round(os.path.getsize("temp_model.pkl") / 1e6, 2)
    os.remove("temp_model.pkl")  # Clean up

    # Log dictionary
    log_dict = {
        "model_name": model_name,
        "target": target_variable,
        "dataset_used": dataset_name,
        "training_samples": X_train.shape[0],
        "test_samples": X_test.shape[0],
        "n_features": X_train.shape[1],
        "input_shape": X_train.shape,
        "mae": round(mae, 2),
        "rmse": round(rmse, 2),
        "r2_score": round(r2, 3),
        "training_time_sec": round(training_time_sec, 2),
        "inference_time_per_sample_sec": inference_time,
        "model_size_mb": model_size_mb,
        "tuning_type": tuning_type,
        "system_info": {
            "platform": platform.system(),
            "platform_version": platform.version(),
            "processor": platform.processor(),
            "cpu_count": psutil.cpu_count(logical=True),
            "gpu_used": gpu_used
        },
        "notes": extra_notes
    }

    # Save JSON
    #os.makedirs("outputs/logs", exist_ok=True)
    json_path = f"outputs/logs/{model_name}_log.json"
    with open(json_path, "w") as f:
        json.dump(log_dict, f, indent=4)

    print(f"✅ JSON log saved at: {json_path}")
    return log_dict

def save_log_to_csv(log_dict, csv_path="../outputs/logs/model_logs.csv"):
    system_info = log_dict.pop("system_info", {})
    flat_log = {**log_dict, **system_info}

    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        df = pd.concat([df, pd.DataFrame([flat_log])], ignore_index=True)
    else:
        df = pd.DataFrame([flat_log])

    df.to_csv(csv_path, index=False)
    print(f"✅ CSV log updated at: {csv_path}")
